In [78]:
import tensorflow as tf
mninst = tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test) = mninst.load_data()
sample = x_test[0]
x_train,x_test = x_train/255.0,x_test/255.0
#入力型を制限
#x_train,x_test = x_train.astype("float32"),x_test.astype("float32")
#テストアイテムを捨てる
#buf = tf.image.encode_png(sample[:, :, tf.newaxis])
#tf.io.write_file("examples/mnist_savedmodel2/sample.png", buf)

In [79]:
class classImage(tf.keras.Model):
  def __init__(self):
    super(classImage,self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
    self.dense1 = tf.keras.layers.Dense(128,activation="relu")
    self.drop = tf.keras.layers.Dropout(0.2)
    self.dense2 = tf.keras.layers.Dense(10)

  def call(self,x):
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.drop(x)
    return self.dense2(x)
    
model = classImage()


In [80]:
class ClassificationModel(tf.keras.Model):
  def __init__(self):
    super(ClassificationModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu',input_shape=(1,28,28))
    self.flatten = tf.keras.layers.Flatten()
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = ClassificationModel()

In [81]:
#損失関数
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
#最適化関数
optimizer = tf.keras.optimizers.Adam()
# 評価指標
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(x,y):
  with tf.GradientTape() as tape:
    predictions = model(x, training=True)
    loss = loss_object(y, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y, predictions)

@tf.function
def test_step(x, t):
    test_predictions = model(x)
    t_loss = loss_object(t, test_predictions)

    test_loss(t_loss)
    test_accuracy(t, test_predictions)


In [82]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
print(train_ds)
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    print(images.shape)
    print(labels.shape)
    train_step(images, labels) #学習

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels) #評価

    template = 'Epoch {}, Loss: {}, Accuracy: {}, test-Loss: {}, test-Accuracy:{}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result()*100))

<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.float64, tf.uint8)>
(32, 28, 28)
(32,)


ValueError: in user code:

    /tmp/ipykernel_268953/3886407337.py:15 train_step  *
        predictions = model(x, training=True)
    /tmp/ipykernel_268953/1604922822.py:10 call  *
        x = self.conv1(x)
    /home/morio/.local/lib/python3.8/site-packages/keras/engine/base_layer.py:1020 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    /home/morio/.local/lib/python3.8/site-packages/keras/engine/input_spec.py:229 assert_input_compatibility
        raise ValueError('Input ' + str(input_index) + ' of layer ' +

    ValueError: Input 0 of layer conv2d_10 is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: (32, 28, 28)


In [ ]:
#確立として解釈できるように、ソフトマックスを介して出力タイプを変換
inputs = tf.keras.Input((28,28),name="input",dtype=tf.float32)
x = model(inputs)
outputs = tf.keras.layers.Softmax(name="output")(x)
probability_model = tf.keras.Model(inputs=inputs, outputs=outputs)